In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from matplotlib.ticker import FuncFormatter

# -----------------------------
# Config
# -----------------------------
DATA_PATH = Path("data_transportation_phd_recruitment.xlsx")  # update if needed
OUTDIR = Path("results_outputs")
FIGDIR = OUTDIR / "figures"
TBLDIR = OUTDIR / "tables"
FIGDIR.mkdir(parents=True, exist_ok=True)
TBLDIR.mkdir(parents=True, exist_ok=True)

COL_ROLE = "Role"
COL_NAME = "Name"
COL_INST = "Institution"
COL_UGDEPT = "UG_Department"

MIN_N_PROGRAM = 10          # for program diversity plot/table
MIN_N_MATCH_PAIRS = 5       # filter small-n categories in matching plot

PLOT_COLOR = "#4C72B0"

# -----------------------------
# Figure saving helper
# -----------------------------
def save_fig(fig, path: Path):
    fig.savefig(
        path,
        dpi=300,
        bbox_inches="tight",
        facecolor="white",
        edgecolor="white",
        transparent=False,
    )

pct = FuncFormatter(lambda y, _: f"{100*y:.0f}%")

# -----------------------------
# UG major grouping
# -----------------------------
def map_ug_department_refined(dept):
    if pd.isna(dept) or str(dept).strip() == "":
        return pd.NA

    d = str(dept).strip()
    d = re.sub(r"\bTranportation\b", "Transportation", d, flags=re.IGNORECASE)
    x = d.lower()

    if "civil" in x:
        return "Civil Engineering"
    if "transport" in x or "traffic" in x:
        return "Transportation Engineering"
    if "mechanical" in x:
        return "Mechanical Engineering"
    if "electrical" in x or x in {"ee"}:
        return "Electrical Engineering"
    if "computer" in x or x in {"cs"} or x == "computing":
        return "Computer Science"
    if "math" in x or "statistics" in x or "stat" in x:
        return "Mathematics / Statistics"
    if "physics" in x:
        return "Physics"
    if "industrial" in x or "operations" in x or x in {"ie"} or "management science" in x:
        return "Industrial / Systems Engineering"
    if "econom" in x:
        return "Economics"

    if any(k in x for k in ["aero", "astronaut", "aircraft", "vehicle", "automotive"]):
        return "Aero / Auto / Vehicle"

    if any(k in x for k in ["urban", "planning", "regional", "geography", "remote sensing"]):
        return "Urban / Geo / Planning"

    if any(k in x for k in [
        "engineering", "robotics", "mechatronics", "materials",
        "manufacturing", "electronic", "information engineering",
        "process", "hydropower", "water", "automation", "control",
        "safety", "logistics", "supply chain"
    ]):
        return "Other Engineering"

    return "Other Non Engineering"

# -----------------------------
# Metrics helpers
# -----------------------------
def simpson_diversity_from_counts(counts):
    counts = np.asarray(counts, dtype=float)
    total = counts.sum()
    if total <= 0:
        return np.nan
    p = counts / total
    return 1.0 - np.sum(p ** 2)

def herfindahl_from_counts(counts):
    counts = np.asarray(counts, dtype=float)
    total = counts.sum()
    if total <= 0:
        return np.nan
    p = counts / total
    return np.sum(p ** 2)

def safe_to_latex(df, path, float_format="%.3f"):
    try:
        df.to_latex(path, index=False, float_format=float_format)
    except Exception:
        df2 = df.copy()
        for c in df2.columns:
            if pd.api.types.is_float_dtype(df2[c]):
                df2[c] = df2[c].map(lambda v: (float_format % v) if pd.notna(v) else "")
        df2.to_latex(path, index=False)

# -----------------------------
# Load
# -----------------------------
xl = pd.ExcelFile(DATA_PATH)
if "People" not in xl.sheet_names:
    raise ValueError(f'Workbook must contain sheet "People". Found: {xl.sheet_names}')

people = pd.read_excel(DATA_PATH, sheet_name="People")
needed_cols = [COL_ROLE, COL_NAME, COL_INST, COL_UGDEPT]
missing = [c for c in needed_cols if c not in people.columns]
if missing:
    raise ValueError(f"Missing columns in People sheet: {missing}. Available: {list(people.columns)}")

people["_role_l"] = people[COL_ROLE].astype("string").str.strip().str.lower()

students = people.loc[people["_role_l"] == "student"].copy()
professors = people.loc[people["_role_l"] == "professor"].copy()

people["UG_Department_Grouped"] = people[COL_UGDEPT].apply(map_ug_department_refined)
students["UG_Department_Grouped"] = students[COL_UGDEPT].apply(map_ug_department_refined)
professors["UG_Department_Grouped"] = professors[COL_UGDEPT].apply(map_ug_department_refined)

print("Rows:", len(people), "Students:", len(students), "Professors:", len(professors))
print("UG_Group coverage among students:", round(float(students["UG_Department_Grouped"].notna().mean()), 3))

# -----------------------------
# Build advisor-student pairs from People blocks
# -----------------------------
def build_pairs_from_people_blocks(df):
    rl = df["_role_l"].astype("string").fillna("").str.strip().str.lower()
    current_prof = None
    pairs = []

    for i in range(len(df)):
        r = df.iloc[i]
        rr = rl.iat[i]

        if rr == "professor":
            current_prof = r
            continue

        if rr == "student" and current_prof is not None:
            pairs.append({
                "Advisor": str(current_prof[COL_NAME]),
                "Advisor_Institution": current_prof[COL_INST],
                "Advisor_UG_Grouped": current_prof.get("UG_Department_Grouped", pd.NA),
                "Student": str(r[COL_NAME]),
                "Student_Institution": r[COL_INST],
                "Student_UG_Grouped": r.get("UG_Department_Grouped", pd.NA),
            })

    return pd.DataFrame(pairs)

pairs = build_pairs_from_people_blocks(people)
pairs.to_csv(TBLDIR / "pairs_extracted_from_people.csv", index=False)
print("Extracted pairs:", len(pairs), "Unique advisors:", pairs["Advisor"].nunique())

# ============================================================
# 5.1 Aggregate Pathways
# ============================================================
agg = (
    students.dropna(subset=["UG_Department_Grouped"])
    .groupby("UG_Department_Grouped")
    .size()
    .rename("n")
    .reset_index()
)
agg["share"] = agg["n"] / agg["n"].sum()
agg = agg.sort_values("n", ascending=False)

agg.to_csv(TBLDIR / "aggregate_ug_grouped.csv", index=False)
safe_to_latex(agg, TBLDIR / "aggregate_ug_grouped.tex")

# Plot: horizontal bar chart to avoid label overlap
agg_top = agg.head(15).copy().sort_values("share", ascending=True)

fig, ax = plt.subplots(figsize=(8.5, 5.5))
ax.barh(
    agg_top["UG_Department_Grouped"].astype(str),
    agg_top["share"].values,
    color=PLOT_COLOR,
)
ax.set_xlabel("Share of students")
ax.set_ylabel("")
ax.xaxis.set_major_formatter(pct)
ax.set_title("Aggregate undergraduate major groups (top 15)")

for y, v in enumerate(agg_top["share"].values):
    ax.text(v + 0.005, y, f"{100*v:.1f}%", va="center", fontsize=8)

ax.set_xlim(0, min(1.0, float(agg_top["share"].max()) + 0.08))
fig.tight_layout()
save_fig(fig, FIGDIR / "fig_aggregate_ug_grouped_top15.png")
plt.close(fig)

# ============================================================
# 5.2 Institutional Variation (Program diversity only)
# ============================================================
prog = students.dropna(subset=["UG_Department_Grouped"]).copy()
prog["Program"] = prog[COL_INST].astype("string").str.strip()

prog_counts = (
    prog.groupby(["Program", "UG_Department_Grouped"])
    .size()
    .rename("n")
    .reset_index()
)

prog_div = (
    prog_counts.pivot_table(index="Program", columns="UG_Department_Grouped", values="n", fill_value=0)
    .reset_index()
)

count_cols = [c for c in prog_div.columns if c != "Program"]
prog_div["n_total"] = prog_div[count_cols].sum(axis=1)
prog_div["D_simpson"] = prog_div[count_cols].apply(lambda r: simpson_diversity_from_counts(r.values), axis=1)
prog_div["H_herfindahl"] = prog_div[count_cols].apply(lambda r: herfindahl_from_counts(r.values), axis=1)
prog_div["effective_groups"] = 1.0 / prog_div["H_herfindahl"].replace(0, np.nan)
prog_div = prog_div.sort_values(["n_total", "D_simpson"], ascending=[False, False])

prog_div.to_csv(TBLDIR / "program_diversity_grouped.csv", index=False)
safe_to_latex(
    prog_div[["Program", "n_total", "D_simpson", "effective_groups"]],
    TBLDIR / "program_diversity_grouped_summary.tex"
)

eligible = prog_div.loc[prog_div["n_total"] >= MIN_N_PROGRAM].copy()
eligible.sort_values("D_simpson", ascending=False).head(10).to_csv(
    TBLDIR / "programs_high_diversity_grouped.csv", index=False
)
eligible.sort_values("D_simpson", ascending=True).head(10).to_csv(
    TBLDIR / "programs_low_diversity_grouped.csv", index=False
)

fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(eligible["D_simpson"].values, bins=15, color=PLOT_COLOR)
ax.set_xlabel("Simpson diversity index")
ax.set_ylabel("Number of programs")
ax.set_title(f"Program level diversity (n >= {MIN_N_PROGRAM})")
fig.tight_layout()
save_fig(fig, FIGDIR / "fig_program_diversity_hist_grouped.png")
plt.close(fig)

# ============================================================
# 5.3 Advisor-Level Recruitment Patterns
# ============================================================
dfp = pairs.dropna(subset=["Advisor", "Student_UG_Grouped"]).copy()

# Advisor dominant share (specialization)
adv_counts = (
    dfp.groupby(["Advisor", "Student_UG_Grouped"])
    .size()
    .rename("n")
    .reset_index()
)
adv_totals = adv_counts.groupby("Advisor")["n"].sum().rename("n_total").reset_index()
adv_counts = adv_counts.merge(adv_totals, on="Advisor", how="left")
adv_counts["share"] = adv_counts["n"] / adv_counts["n_total"]

adv_wide = (
    adv_counts.pivot_table(index="Advisor", columns="Student_UG_Grouped", values="share", fill_value=0.0)
    .reset_index()
)
adv_wide = adv_wide.merge(adv_totals, on="Advisor", how="left")

share_cols = [c for c in adv_wide.columns if c not in ["Advisor", "n_total"]]
adv_wide["Top_share"] = adv_wide[share_cols].max(axis=1)

adv_wide.to_csv(TBLDIR / "advisor_topshare_grouped.csv", index=False)

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(adv_wide["Top_share"].values, bins=20, color=PLOT_COLOR)
ax.axvline(0.5, linestyle="--", linewidth=1)
ax.set_xlabel("Advisor dominant student group share")
ax.set_ylabel("Number of advisors")
ax.set_title("Advisor level specialization (dominant share)")
fig.tight_layout()
save_fig(fig, FIGDIR / "fig_advisor_topshare_hist_grouped.png")
plt.close(fig)

# Matching by advisor background
dfm = dfp.dropna(subset=["Advisor_UG_Grouped", "Student_UG_Grouped"]).copy()
dfm["Match"] = (dfm["Advisor_UG_Grouped"] == dfm["Student_UG_Grouped"]).astype(int)

match_by_adv = (
    dfm.groupby("Advisor_UG_Grouped")["Match"]
    .agg(match_rate="mean", n_pairs="size")
    .reset_index()
    .sort_values("n_pairs", ascending=False)
)

match_by_adv.to_csv(TBLDIR / "matching_rate_by_advisor_grouped_full.csv", index=False)

plot_match = match_by_adv.loc[match_by_adv["n_pairs"] >= MIN_N_MATCH_PAIRS].copy()
plot_match.to_csv(TBLDIR / "matching_rate_by_advisor_grouped_plot.csv", index=False)

# Plot: horizontal bar chart to avoid label overlap
plot_match = plot_match.sort_values("match_rate", ascending=True)

fig, ax = plt.subplots(figsize=(8.5, 5.0))
ax.barh(
    plot_match["Advisor_UG_Grouped"].astype(str),
    plot_match["match_rate"].values,
    color=PLOT_COLOR,
)
ax.set_xlabel("Match rate")
ax.set_ylabel("")
ax.xaxis.set_major_formatter(pct)
ax.set_title(f"Student advisor major group matching (n_pairs >= {MIN_N_MATCH_PAIRS})")

for y, (_, row) in enumerate(plot_match.reset_index(drop=True).iterrows()):
    ax.text(float(row["match_rate"]) + 0.01, y, f"n={int(row['n_pairs'])}", va="center", fontsize=8)

ax.set_xlim(0, 1.0)
fig.tight_layout()
save_fig(fig, FIGDIR / "fig_matching_rate_by_advisor_grouped.png")
plt.close(fig)

print("Done. Outputs saved to:", OUTDIR.resolve())


Rows: 449 Students: 322 Professors: 127
UG_Group coverage among students: 0.863
Extracted pairs: 322 Unique advisors: 78
Done. Outputs saved to: C:\Users\andre\OneDrive\Documents\UG_major_paper\results_outputs
